<a href="https://colab.research.google.com/github/HackyRoot/SeeFood/blob/master/SeeFood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SeeFood

## Import packages

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Load data

Source: https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog

In [11]:
!wget https://raw.githubusercontent.com/HackyRoot/SeeFood/master/data.zip
!unzip data.zip
!rm data.zip*
!ls

--2020-06-05 03:53:48--  https://raw.githubusercontent.com/HackyRoot/SeeFood/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46841867 (45M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  44.67M  62.3MB/s    in 0.7s    

2020-06-05 03:53:49 (62.3 MB/s) - ‘data.zip’ saved [46841867/46841867]

Archive:  data.zip
replace data/val/hot_dog/527633.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
data  sample_data


In [0]:
data_dir = 'data/'

In [13]:
!ls data/train

hot_dog  not_hot_dog


In [0]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

In [0]:
train_hotdog_dir = os.path.join(train_dir, 'hot_dog')
val_hotdog_dir = os.path.join(val_dir, 'hot_dog')

train_not_hotdog_dir = os.path.join(train_dir, 'not_hot_dog')
val_not_hotdog_dir = os.path.join(val_dir, 'not_hot_dog')

## Understand the data

In [0]:
num_train_hd = len(os.listdir(train_hotdog_dir))
num_val_hd = len(os.listdir(val_hotdog_dir))

num_train_nhd = len(os.listdir(train_not_hotdog_dir))
num_val_nhd = len(os.listdir(val_not_hotdog_dir))

total_train = num_train_hd + num_train_nhd
total_val = num_val_hd + num_val_nhd

In [19]:
print('total training hotdog images: ', num_train_hd)
print('total validation hotdog images: ', num_val_hd)

print('total training not-hotdog images: ', num_train_nhd)
print('total validation not-hotdog images: ', num_val_nhd)

print('-----------------------------------')

print('total training images: ', total_train)
print('total validation images: ', total_val)

total training hotdog images:  397
total validation hotdog images:  101
total training not-hotdog images:  397
total validation not-hotdog images:  102
-----------------------------------
total training images:  794
total validation images:  203


## Data preparation

In [0]:
batch_size = 24
epochs = 15
IMG_SIZE = 224

In [0]:
train_image_generator = ImageDataGenerator(rescale=1/255)
validation_image_generator = ImageDataGenerator(rescale=1/255)

In [23]:
train_data_gen = train_image_generator.flow_from_directory(batch_size = batch_size, 
                                          directory=train_dir,
                                          shuffle=True,
                                          target_size=[IMG_SIZE, IMG_SIZE],
                                          class_mode='binary')

Found 794 images belonging to 2 classes.


In [24]:
val_image_generator = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                                     directory=val_dir,
                                                                     shuffle=True,
                                                                     target_size=[IMG_SIZE, IMG_SIZE],
                                                                     class_mode='binary')

Found 203 images belonging to 2 classes.
